# 1. Given the final code example in this section with kernel size (3,5), padding (0,1), and stride (3,4), calculate the output shape to check if it is consistent with the experimental result.

$[x_w-k_w+2*pad_w+s_w]//s_w=(8-3+3)//3=2$

$[x_h-k_h+2*pad_h+s_h]//s_h=(8-5+2+4)//4=2$

In [16]:
import torch
import torch.nn as nn
def comp_conv2d(conv2d, X):
    # (1, 1) indicates that batch size and the number of channels are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Strip the first two dimensions: examples and channels
    return Y.reshape(Y.shape[2:])

# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=(3,5), padding=(0,1), stride=(3,4))
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

/home/jovyan/.local/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 2])

# 2. For audio signals, what does a stride of 2 correspond to?

In the context of audio signals and convolutional operations, a stride of 2 corresponds to how much you move the convolutional kernel across the input signal between each computation of the convolution. More specifically, a stride of 2 means that the kernel will "step" or "skip" every 2 units (samples) along the input signal.

When performing convolution with a larger stride, the resulting output will have a reduced length compared to the input. This can be particularly useful for downsampling or reducing the dimensionality of the data. It effectively "compresses" the information in the signal.

For audio signals, a stride of 2 could be used to:

1. **Downsample the Audio**: If you're dealing with audio data and want to reduce its sampling rate or compress the information, you can apply convolution with a stride of 2. This can help in reducing computational load and capturing only essential features.

2. **Feature Extraction**: Strided convolutions can also be useful in extracting important features from audio signals. The higher stride can capture higher-level characteristics of the audio signal, which might be relevant for certain applications like music genre classification or speech recognition.

3. **Pooling Operations**: Strided convolutions are commonly used in conjunction with pooling layers (like MaxPooling) to further reduce the dimensionality of the data. Pooling helps retain the most important features while discarding less relevant information.

Remember that the choice of stride depends on your specific application and goals. A stride of 2 can lead to more aggressive downsampling or feature extraction, while smaller strides (like 1) retain more information but may be computationally more expensive.

# 3. Implement mirror padding, i.e., padding where the border values are simply mirrored to extend tensors.

In [19]:
import torch

def mirror_padding(X, padding_size):
    # Check if padding_size is a tuple or single value
    # if isinstance(padding_size, int):
    #     padding_size = (padding_size, padding_size)
    # elif len(padding_size) != 2:
    #     raise ValueError("padding_size should be a single value or a tuple of two values")
    X = X.reshape((1, 1) + X.shape)
    # Get dimensions of input tensor
    batch_size, num_channels, height, width = X.size()

    # Pad along height and width dimensions
    padded_tensor = torch.nn.functional.pad(X,
                                            (padding_size[1], padding_size[1], padding_size[0], padding_size[0]),
                                            mode='reflect')

    return padded_tensor

X = torch.range(1,64).reshape(8,8)
padded_tensor = mirror_padding(X, (1,1))
print("Original tensor:")
print(X)
print("Padded tensor:")
print(padded_tensor)


Original tensor:
tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12., 13., 14., 15., 16.],
        [17., 18., 19., 20., 21., 22., 23., 24.],
        [25., 26., 27., 28., 29., 30., 31., 32.],
        [33., 34., 35., 36., 37., 38., 39., 40.],
        [41., 42., 43., 44., 45., 46., 47., 48.],
        [49., 50., 51., 52., 53., 54., 55., 56.],
        [57., 58., 59., 60., 61., 62., 63., 64.]])
Padded tensor:
tensor([[[[10.,  9., 10., 11., 12., 13., 14., 15., 16., 15.],
          [ 2.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  7.],
          [10.,  9., 10., 11., 12., 13., 14., 15., 16., 15.],
          [18., 17., 18., 19., 20., 21., 22., 23., 24., 23.],
          [26., 25., 26., 27., 28., 29., 30., 31., 32., 31.],
          [34., 33., 34., 35., 36., 37., 38., 39., 40., 39.],
          [42., 41., 42., 43., 44., 45., 46., 47., 48., 47.],
          [50., 49., 50., 51., 52., 53., 54., 55., 56., 55.],
          [58., 57., 58., 59., 60., 61., 62., 63., 64., 63.],
         

/tmp/ipykernel_2092715/57150614.py:20: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  X = torch.range(1,64).reshape(8,8)


# 4. What are the computational benefits of a stride larger than 1?

Using a stride larger than 1 in convolutional operations can bring several computational benefits:

1. **Reduced Computational Load**: Larger strides result in fewer convolutional operations. This reduces the number of calculations required to compute the output feature maps, which can significantly speed up the computation, especially for large input tensors.

2. **Downsampling**: A larger stride effectively reduces the spatial resolution of the output feature maps. This can be useful for downsampling the data, which is often desired in various scenarios like image classification, where high-resolution details might not be necessary at all stages of processing.

3. **Reduced Memory Usage**: With a larger stride, the output feature maps have smaller dimensions, leading to reduced memory usage. This can be particularly helpful when dealing with memory-intensive tasks like training deep neural networks.

4. **Increased Receptive Field**: A larger stride increases the receptive field of each output unit. This means that each output unit considers a wider region of the input, which can be advantageous for capturing more abstract and higher-level features.

5. **Simplified Architecture**: Using larger strides can simplify the architecture by reducing the number of layers needed to achieve a certain level of downsampling. This can lead to shallower networks that are computationally more efficient.

6. **Feature Reduction**: Larger strides can suppress fine-grained features and focus on more prominent features, which might be beneficial for noise reduction or highlighting key patterns.

7. **Regularization**: Strided convolutions can act as a form of regularization by enforcing a form of local pooling, making the model less likely to overfit the training data.

However, it's important to note that using larger strides also has potential downsides:

1. **Reduced Spatial Information**: Larger strides result in a loss of spatial information. This might be undesirable if you need fine-grained details for your task, such as object localization or segmentation.

2. **Less Localization Accuracy**: If you're performing tasks that require precise localization, such as object detection, larger strides might lead to reduced localization accuracy.

3. **Smaller Receptive Field**: Larger strides can lead to smaller effective receptive fields of neurons, which might hinder capturing long-range dependencies.

The choice of stride depends on the task, the architecture, and the nature of the data. Larger strides are often used in early layers of deep networks to quickly reduce dimensions, and then smaller strides are used in subsequent layers to capture finer details.

# 5. What might be statistical benefits of a stride larger than 1?

Using a stride larger than 1 in convolutional operations can have statistical benefits in certain scenarios:

1. **Statistical Independence**: A larger stride can lead to more statistically independent outputs. When the stride is large, the receptive fields of adjacent output units have less overlap, resulting in less correlated responses. This can be advantageous when you want to capture diverse and distinct features across different regions of the input.

2. **Reduced Overlapping Information**: With a larger stride, neighboring output units have fewer overlapping input regions. This can help in reducing redundancy in the extracted features, allowing the model to focus on unique and salient aspects of the data.

3. **Statistical Diversity**: A larger stride encourages the network to capture diverse features by ensuring that different output units focus on different parts of the input. This can lead to a richer and more diverse set of learned features.

4. **Noise Suppression**: Larger strides can act as a form of noise suppression. By capturing only key information from each receptive field, the network may become less sensitive to small-scale noise in the input data.

5. **Dimensionality Reduction**: With a larger stride, the spatial dimensions of the output feature maps are reduced. This can be seen as a form of dimensionality reduction, which can help in capturing the most important information while discarding less significant details.

6. **Feature Abstraction**: Larger strides encourage the network to focus on higher-level abstractions. The network is forced to capture the most essential features using a smaller number of operations, which can lead to more efficient and effective representations.

It's important to note that the choice of stride depends on the specific data, task, and architecture. While larger strides can provide statistical benefits, they also come with downsides, such as reduced spatial resolution and potential loss of fine-grained details. Stride selection should be based on a trade-off between computational efficiency, feature capture, and task requirements.

# 6. How would you implement a stride of $1/2$? What does it correspond to? When would this be useful?

To implement a stride of 1/2 using PyTorch, you can use the transposed convolution operation, also known as the fractionally strided convolution or deconvolution operation. Here's how you can achieve this:

In [23]:
import torch
import torch.nn as nn

# Input tensor
input_tensor = torch.randn(1, 3, 16, 16)  # Example: 1 batch, 3 channels, 16x16 input

# Transposed convolution layer with stride of 1/2
transposed_conv = nn.ConvTranspose2d(in_channels=3, out_channels=3, kernel_size=3, stride=2, padding=1)

# Applying the transposed convolution
output_tensor = transposed_conv(input_tensor)

# Print the shapes of input and output tensors
print("Input tensor shape:", input_tensor.shape)
print("Output tensor shape:", output_tensor.shape)


Input tensor shape: torch.Size([1, 3, 16, 16])
Output tensor shape: torch.Size([1, 3, 31, 31])


Transposed convolution effectively "upsamples" the input tensor by creating larger output feature maps. The stride of 2 in the transposed convolution results in an output tensor with dimensions that are approximately twice as large as the input tensor along each dimension.

Keep in mind that using transposed convolution requires careful tuning and consideration of the output tensor's dimensions. Additionally, fractional stride operations like this can introduce artifacts, such as checkerboard patterns, so post-processing techniques like bilinear interpolation or sub-pixel convolution might be necessary for better results.